In [10]:
import numpy as np
from CoDIAC import jalviewFunctions, featureTools
import pandas as pd

# Number of SH2 domain positions that must have a feature to be considered conserved
LIGAND_CONSERVATION = 8

# These aren't currently being used, but these might be considered as the cutoff for conserved, based on total numbers
PHOSPHOTYROSINE = 5
PHOSPHOSERINE = 5
UBIQUITINATION = 5
PHOSPHOTHREONINE = 3
ACETYLATION = 3

aln_file = 'Data/Uniprot_Reference/alignment/SH2_IPR000980_promals3d.fasta'
aln_mapping_dict, aln = featureTools.return_position_mapping_dict(aln_file)

# Load the domain domain interactions
domain_file = 'Data/Features/interDomain/SH2_IPR000980_interDomain_phospholipid.feature'  #will treat phospholipid as a feature like domain interaction interface for the purposes of interesecting
feature_colors, domain_int_dict = jalviewFunctions.return_features_from_file(domain_file)

# Will also compare to the ligand binding sites
ligand_file = 'Data/Features/ligand/SH2_IPR000980_Ligand.feature'
feature_colors, ligand_dict = jalviewFunctions.return_features_from_file(ligand_file)
ligand_ann = featureTools.return_feature_ann_dict(aln, ligand_dict)
ligand_ann_conserved = ligand_ann['Ligand'].sum() >= LIGAND_CONSERVATION

#load all of the PTM features
feature_dir = 'Data/Features/'
ptm_dir = feature_dir + 'PTMS_all/'
mods = ['N6-acetyllysine', 'Phosphoserine', 'Phosphothreonine', 'Phosphotyrosine', 'Ubiquitination',
        'Phosphoserine_Phosphothreonine']
ptm_dict = {}
ptm_colors = {}
ptm_annotations_dict = {}
for mod in mods:
    file_root = ptm_dir + 'SH2_IPR000980_' + mod
    feature_file = file_root + '.feature'
    ann_file = file_root + '.ann'
    if mod == 'Phosphoserine_Phosphothreonine':
        mod = 'Phosphorylation(ST)'
    ptm_colors[mod], ptm_dict[mod] = jalviewFunctions.return_features_from_file(feature_file)
    ptm_annotations_dict[mod] = featureTools.return_feature_ann_dict(aln, ptm_dict[mod])


### Produce a PTM-centric information table, based on contacts of ligand and domain-domain interfaces

In [11]:

#these are direct intersections between PTMs of type mod and domain and ligand binding sites
feature_intersection_domain_dict = {}
feature_intersection_ligand_dict = {}
for mod in ptm_dict.keys():
    feature_intersection_domain_dict[mod], headers_no_contacts = featureTools.return_intersection(ptm_dict[mod],
                                                                                                  domain_int_dict)  #update this to return something, even if empty
    feature_intersection_ligand_dict[mod], headers_no_contacts = featureTools.return_intersection(ptm_dict[mod],
                                                                                                  ligand_dict)



In [13]:

for mod in ptm_dict.keys():
    df = create_report_for_mod(mod, ptm_dict, ptm_annotations_dict, aln_mapping_dict, ligand_ann_conserved,
                               feature_intersection_domain_dict)
    df.to_csv('Data/Reports/SH2_IPR000980_%s_summary.csv' % mod, index=False)

In [9]:
# Write a readme file that documents what we did, what it means, and what the cutoffs are
from datetime import datetime

# current date and time
now = datetime.now()

readme_file = 'Data/Reports/FeatureAnalysis.md'
f = open(readme_file, 'w')
f.write('## Feature Analysis\n')
f.write('This document describes the features that were analyzed for the SH2 domain as reported in this directory\n')
f.write('These reports were generated by the CoDIAC package on: %s\n' % now.strftime("%m/%d/%Y %H:%M:%S"))
f.write('### Analysis Reports (CSV files)\n')
f.write('Features were analyzed with regard to:\n')
f.write('1. The proximity to a conserved ligand binding sites\n')
f.write('2. The direct intersection with domain-domain interactions\n')
f.write('3. The number of other features of that type in the alignment\n')
f.write('### Feature Cutoffs and files used\n')
f.write(
    'Ligand binding was considered conserved if >= %d residues in that alignment position were in the ligand binding features\n' % (
        LIGAND_CONSERVATION))
f.write('The following files were used:\n')
f.write('1. %s\n' % aln_file)
f.write('2. %s\n' % domain_file)
f.write('3. %s\n' % ligand_file)
f.write('4. PTMs in the following directory %s\n' % ptm_dir)
f.write('### Analysis Report Columns\n')
f.write('1. header: the fasta header of the sequence\n')
f.write('2. Uniprot_ID: the uniprot ID of the sequence\n')
f.write('3. gene_name: the gene name of the sequence\n')
f.write('4. SH2 number: the SH2 domain number (1 if N-terminal and 2 if C-terminal domain in 2 SH2 domain proteins\n')
f.write('5. protein_residue: the residue in the protein sequence numbering\n')
f.write('6. feature_position: the position in the domain, as printed in the feature file, relative to SH2 start\n')
f.write('7. alignment_postion: the position in the alignment\n')
f.write('8. number_mods_at_position: the number of modifications of this type at that position\n')
f.write(
    '9. number_conserved_ligand_residues_nearby: the number of residues in the ligand binding site at that position within +/- 2 amino acids away\n')
f.write(
    '10. number_residues_from_conserved_ligand: the number of residues from the conserved ligand binding site. If 0, it falls directly on a ligand interface.\n')
f.write(
    '11. domain_interaction: the domain interaction at that position, if it intersects directly with a domain-domain interface. We also use this column to indicate if it interacts with a residue identified as interacting with phospholipids\n')
f.write('### PTM Analysis\n')
f.write('The following PTMs were analyzed:\n')
f.write('1. N6-acetyllysine\n')
f.write('2. Phosphoserine\n')
f.write('3. Phosphothreonine\n')
f.write('4. Phosphotyrosine\n')
f.write('5. Ubiquitination\n')
f.write('6. Phosphorylation (Phosphoserine and Phosphothreonine combined)\n')
f.close()






In [12]:
def create_report_for_mod(mod, ptm_dict, ptm_annotations_dict, aln_mapping_dict, ligand_ann_conserved,
                          feature_intersection_domain_dict):
    ptm_features = ptm_dict[mod]
    ptm_annotations = ptm_annotations_dict[mod]
    sum_ann = ptm_annotations[mod].sum()
    number_dict = {}
    ligand_conserved_window = {}
    ligand_distance_dict = {}

    # Collecting information on PTMs and ligand binding sites
    for header in ptm_features:
        number_dict[header] = {}
        ligand_conserved_window[header] = {}
        ligand_distance_dict[header] = {}
        aln_map = aln_mapping_dict[header]
        for pos in ptm_features[header]:
            pos_index = int(pos) - 1  #have to translate to 0-based index
            aln_pos = aln_map[pos_index]  #this is the position in the alignment, 0-based
            ptm_number = sum_ann[int(aln_pos)]
            number_dict[header][pos] = {'aln_pos': aln_pos + 1, 'num_mods': ptm_number}
            ligand_conserved_window[header][pos] = ligand_ann_conserved[
                                                   aln_pos - 2:aln_pos + 2].sum()  #total number of conserved interaction residues in a window of 5 residues
            #let's record the furthest distance it is to a conserved ligand binding sit
            ligand_distance_dict[header][pos] = min(abs(aln_pos - np.where(ligand_ann_conserved)[0]))

    df = pd.DataFrame(columns=['header', 'Uniprot_ID', 'gene_name', 'SH2 number', 'protein_residue', 'feature_position',
                               'alignment_postion', 'number_mods_at_position',
                               'number_conserved_ligand_residues_nearby', 'number_residues_from_conserved_ligand',
                               'domain_interaction'])
    for header in ptm_features:
        uniprot_id, name, number, start, stop = header.split('|')
        for pos in ptm_features[header]:
            arr_temp = feature_intersection_domain_dict[mod][header][pos]
            string_domain = ';'.join(arr_temp)
            dict_temp = {'header': header, 'Uniprot_ID': uniprot_id, 'gene_name': name, 'SH2 number': number,
                         'protein_residue': int(start) + int(pos) - 1, 'feature_position': pos,
                         'alignment_postion': number_dict[header][pos]['aln_pos'],
                         'number_mods_at_position': number_dict[header][pos]['num_mods'],
                         'number_conserved_ligand_residues_nearby': ligand_conserved_window[header][pos],
                         'number_residues_from_conserved_ligand': ligand_distance_dict[header][pos],
                         'domain_interaction': string_domain}
            df_temp = pd.DataFrame.from_dict(dict_temp, orient='index').T
            df = pd.concat([df, df_temp])
    return df